In [1]:
import sys  
sys.path.insert(0, '../search-engine')
from utils import *
import pandas as pd
from igraph import *
import implicit
from scipy.sparse import csr_matrix
from sklearn.metrics import ndcg_score
from copy import deepcopy


Bad key "text.kerning_factor" on line 4 in
/usr/local/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


# NB3 - Link analysis

## 1. Load data 
Read tweets from data directory.

In [2]:
data = get_tweets(None, None, mode = "read", data_directory = '../data/')

## 2. Graph creation

Graph will be created using `igraph` library. The graph will contain only users that have retweeted tweets from other users that also appear in the data collection. Main characteristics are: 
- Direted graph: A -> B means user A retweeted a tweet from user B. 
- Nodes: Users
- Edges: Retweets. Not weighted graph: meaning that, if user A retweeted three times user B, the number of retweets will not be represented in the graph, only the connection between users. 

NOTE: `igraph` graphs do not contain labels for nodes, therefore, there needs to be a mapping between users' id and the id of the node in the graph. 

In [3]:
# Mapping between user id and node id in the graph.
ids_mapping = {}
counter = 0
for tweet in data:
    if "retweeted_status" in tweet.keys():
        if tweet["user"]["id"] not in ids_mapping.keys():
            ids_mapping[tweet["user"]["id"]] = counter            
            counter += 1

Creation of a dictionary containing as key the node id of the users that retweeted at least a tweet and as value, the node id of the users that wrote the original tweets.

An additional dictionary with the hashtags of each tweet (having as key the node id of the users) that will be needed later. 

In [4]:
# Creation of a dictionaries
retweets = {}
hashtags = {} 

for i, tweet in enumerate(data): 
    try: 
        # Store relations between users that have retweeted at least one tweet
        id2 = ids_mapping[tweet["retweeted_status"]["user"]["id"]]        
        id1 = ids_mapping[tweet["user"]["id"]]
        try: retweets[id1].append(id2)
        except: retweets[id1] = [id2]
        
        if id1 not in hashtags.keys(): hashtags[id1] = []
        if id2 not in hashtags.keys(): hashtags[id2] = []
        
        # Store hashtags list for each user for personalized score
        for hashtag in tweet["entities"]["hashtags"]:
            hashtags[id1].append(hashtag["text"])
            hashtags[id2].append(hashtag["text"])
    except: 
        continue

In [5]:
# Transform hashtags dictionary into a list for the creation of the graph
edgelist = []
for node in retweets:
    for i in retweets[node]:
        edgelist.append((node, i))
        
edgelist = list(set(edgelist)) # Remove duplicated links (since the graph is not weighted)

Creation of the graph. 

In [6]:
g = Graph(n = len(ids_mapping))
g.add_edges(edgelist)

## 3. Networks based predictions
Prepare the dataset: split graph into train and test, 80-20% respectively. 

In [7]:
p = 0.2       # fraction of edges to select as test-set
N = len(g.es) # graphsize

all_idxs = range(N) # idxs of all the edges

# sample randomly test nodes 
np.random.seed(0)
test_idxs = np.random.choice(a=all_idxs, size=int(p*N),replace=False)

# Create the train-set and ground truth for the test set 
ground_truth = set() # links between nodes of the test set
trainset = set()     # links between nodes of the train set

for idx, one_edge in enumerate(g.es):    
    n1 = one_edge.source
    n2 = one_edge.target

    if idx in test_idxs:
        ground_truth.add((n1, n2))
    else:
        trainset.add((n1, n2))

### Distance 2 
The goal of this part of the project is to predict the probability of having a node for those users at distance 2. 

Therefore, we need to consider those nodes that are at distance 2, to predict if there's a node between them. 

In [8]:
# Select nodes at distance 2 (they can also may be at distance 1)
def find_nodes_at_distance_2(graph):
    all_potential_recommendations = set()
    
    for n1 in graph.vs:
        # all the nodes at distance 1 and distance 2
        nodes_at_most_distant_2 = set(graph.neighborhood(n1, order = 2))
            
        if len(nodes_at_most_distant_2) > 0:
            for n2 in nodes_at_most_distant_2: 
                n1_index = n1.index
                if n2 != n1_index: 
                    a = min(n2, n1_index)
                    b = max(n2, n1_index)
                    all_potential_recommendations.add((a,b))
    return list(set(all_potential_recommendations))

# Get nodes at distance 2 in test set
dist_2 = find_nodes_at_distance_2(g)
test_nodes = [(u,v) for u, v in dist_2 if (u in test_idxs) and (v in test_idxs)]

Create a dataframe containing the ground truth for the test set. 0 means they have no direct link, while 1 means they are connected. 

NOTE: there is a path of length at most 2 between the pairs of nodes we are considering.

In [9]:
# Create dataframe with ground truth
test_list = []
for u, v in test_nodes: 
    if (u, v) in ground_truth: test_list.append((u,v,1))
    else: test_list.append((u,v,0))
        
test_df = pd.DataFrame(test_list, columns = ["u", "v", "ground_truth"])
test_df.head()

,u,v,ground_truth
0,5951,13583,0
1,3908,10085,0
2,325,8039,0
3,1156,6255,0
4,14735,14835,0


## 3.1. Adamic-Adar (ADA)
In this section we will use Adamic-Adar to compute the probability of having a direct link between all nodes that are at most at distance 2. 
This algorithm does not require a training set, it can be directly applied to the test set. 

In [10]:
def compute_ADA(u,v, graph):
    # set of neighbors of u
    outlinks_from_u = set(graph.neighbors(u))
    # set of neighbors of v
    inlinks_to_v = set(graph.neighbors(v))
    
    # set Z of neighbors of both
    bridges = outlinks_from_u.intersection(inlinks_to_v)
    
    # degree of nodes in set Z
    deg_ = [graph.degree(node) for node in bridges]
    
    # computing the reciprocal in log-scale
    out = [1./np.log2(dd+1) for dd in deg_]

    return sum(out)

In [11]:
# Predict the probability
pred_ADA = []
# Predictions
for i, row in test_df.iterrows():
    pred = compute_ADA(int(row["u"]), int(row["v"]), g)
    pred_ADA.append(pred)
    
test_df["ADA"] = pred_ADA # Append probabilities to the dataframe
test_df.head()

,u,v,ground_truth,ADA
0,5951,13583,0,0.095358
1,3908,10085,0,0.095358
2,325,8039,0,0.113583
3,1156,6255,0,0.172195
4,14735,14835,0,0.127667


## 3.2. Alternating Least Squares (ALS)
This algorithm has been tested using `implicit` libary. It requires an adjacency matrix (a sparse one). And the approximated predictions are obtained from the dot product of the low-dimensionality vectors obtained from the matrix factorization performed by the model.

In [ ]:
# Get the adjacency matrix data
M = g.get_adjacency().data
M = csr_matrix(M)

In [ ]:
# Declare the model ALS
model = implicit.als.AlternatingLeastSquares(factors=10, calculate_training_loss=True, iterations=5)

# Train the model on a sparse matrix
model.fit(M)

In [ ]:
# Link prediction
def predict_ALS(testset, model):
    all_predictions = []
    for n1, n2, w in testset:
        # take low-dimensional vectors returned by the matrix factorization
        array_n1 = model.user_factors[n1, :]
        array_n2 = model.item_factors[n2, :]

        # approximation for the edge score
        one_p = np.dot(array_n1, array_n2)
        all_predictions.append(one_p)
    return all_predictions

In [ ]:
# Predict
all_predictions = predict_ALS(test_df[["u", "v", "ground_truth"]].values, model)

test_df["ALS"] = all_predictions # Append predictions to the dataframe

In [ ]:
test_df.head()

## 3.3. Personalized PageRank (PPageRank)

The library `igraph` has implemented a personalized page rank computation. In personalized page rank, the starting node needs to be specified.


For each pair of nodes (A, B) in the graph, the probability of having a link between them is computed by computing the PPageRank starting from A and getting the probability for B. 

In [ ]:
# Since PPageRank is performed given a graph, all edges from outside the testset should
# be dropeed. 
new_graph = deepcopy(g)
new_graph.delete_vertices([i for i in range(len(ids_mapping)) if i not in test_idxs])

In [ ]:
def ppage_rank(graph, test_df):
    probab = []
    # For each pair of nodes, compute their probability.
    for u, v in test_df[["u", "v"]].values: 
        probab.append(graph.personalized_pagerank(reset_vertices=u)[v])
    return probab

In [ ]:
test_df["PPageRank"] = ppage_rank(g, test_df)

In [ ]:
test_df

## 3.4. Personalized score
**Main idea**

Improve ADA algorithm by adding a partial score to the one returned by ADA algorithm. 
The partial score considers information from the texts: the hashtags that each user has used. By doing so, we can relate the users having into account their common interests. 

The dictionary contianing the hashtags used by each user has been created at the beginning of section 2. 


**Personalized score**

\begin{equation*} \mathbf{diversity\_score = ADA + \lambda * partial\_score} \end{equation*}
Where: 

- $\lambda = 1/3 * max(ADA)$ regulating the impact on the score 

- $ partial\_score = Jaccard(hashtags\_userA, hashtags\_userB)$

By considering the Jaccard similarity between the sets of hashtags used by both users, we are selecting the keywords that the users themselves have selected to summarize the content they are posting. 

In [ ]:
# Jaccard similarity
def compute_Jaccard(u,v):
    if len(set(u).union(set(v))) == 0:
        return 0.0
    return len(set(u).intersection(set(v)))/len(set(u).union(set(v)))

In [ ]:
personalized_score = []
lamb = 1/3
max_ADA = test_df["ADA"].max()

# Compute personalized score
for i, row in test_df.iterrows():
    pred = compute_Jaccard(hashtags[int(row["u"])], hashtags[int(row["v"])])
    personalized_score.append(row["ADA"] + lamb * max_ADA * pred)
    
# Add it to the test dataframe    
test_df["Personalized"] = personalized_score 
test_df.head()

## 4. Predictions
Top-10 predictions for each algorithm. 

In [ ]:
def get_topk (scores, column, topk = 10):
    scores = scores.sort_values(by=column, ascending = False)
    return scores[:topk]

In [ ]:
algorithms = ["ADA", "ALS", "PPageRank", "Personalized"]
for algorithm in algorithms:
    print("\n\nTop-10 recommendations from {} algoritm".format(algorithm))
    print(get_topk(test_df, algorithm)[["u", "v", algorithm, "ground_truth"]].head())

## 5. Scores comparison - nDCG and precision@k

**nDCG**


nDCG is used to asses the performance of ranking algorithms for categorical scores. It assumes that the added value of an element decreases as it is placed in lower positions of the ranking. 

To compute nDCG, we used a scoring from 0 to 3, being 3 the best score. 
For the ground trugh, all 1 (indicating a link) have been replaced with the top score 3, while 0s (indication no direct link) have been left as the worst score 0. 

For the output of the algorithms, they have been mapped into the scale from 0 to 3 taking into account their minimum and maximum values.

In [ ]:
#Transform continuous scores to categorical ones (from 0 to 3) being 3 the best score. 

def compute_intervals(df, column, n):
    intervals = []
    minimum = df[column].min()
    maximum = df[column].max()
    interval_length = (maximum - minimum) / (n + 1)
    
    for i in range(n + 1):
        intervals.append((minimum + interval_length * i, minimum + interval_length * (i+1)))
    return intervals

def substitution (x, intervals):
    for i, pos in enumerate(intervals): 
        if x >= pos[0] and x <= pos[1]:
            return i
        
ranked_scores = test_df.copy()

ranked_scores["ground_truth"] = ranked_scores["ground_truth"].apply(lambda x: 3 if x == 1 else 0)
ranked_scores["ADA"] = ranked_scores["ADA"].apply(lambda x: substitution(x, compute_intervals(ranked_scores,"ADA",3)))
ranked_scores["ALS"] = ranked_scores["ALS"].apply(lambda x: substitution(x, compute_intervals(ranked_scores,"ALS",3)))
ranked_scores["PPageRank"] = ranked_scores["PPageRank"].apply(lambda x: substitution(x, compute_intervals(ranked_scores,"PPageRank",3)))
ranked_scores["Personalized"] = ranked_scores["Personalized"].apply(lambda x: substitution(x, compute_intervals(ranked_scores,"Personalized",3)))

ranked_scores.sort_values(by="ground_truth", ascending=False).head()

nDCG scores computation for each algorithm

In [ ]:
def dcg_at_k(y_true, y_score,  k=10):
    order = np.argsort(y_score)[::-1] # get the list of indexes of the predicted score sorted in descending order.
    y_true = np.take(list(y_true), order[:k]) # sort the actual relevance label of the documents based on predicted score(hint: np.take) and take first k.
    gain = 2**(y_true) - 1 # Compute gain (use formula 7 above)
    discounts = np.log2(np.arange(len(y_true)) + 2) # Compute denominator
    return np.sum(gain / discounts) #return dcg@k


def ndcg_at_k(y_true, y_score, k=10):    
    dcg_max = dcg_at_k(y_true, y_true, k) # Ideal dcg
    if not dcg_max:
        return 0
    return np.round(dcg_at_k(y_true, y_score, k) / dcg_max,4)  # return ndcg@k

def compute_ndcg(df, column, topk = 20): 
    sorted_ = df.sort_values(by = column, ascending = False)
    return ndcg_at_k(sorted_["ground_truth"], sorted_[column], topk)

In [ ]:
for algorithm in algorithms:
    print("nDCG score ranking with {} algorithm: {}".format(algorithm, compute_ndcg(ranked_scores, algorithm)))

**Precision@k**

p@20 score has also been used to compare the performance of the algorithms, by checking whether the top-20 link connections recommended by each algorithm are in the actual ground truth. 

In [ ]:
def precision_at_k (scores, column, topk = 20):
    scores = get_topk(scores, column, topk)
    return (scores["ground_truth"].sum())/(topk)

In [ ]:
k = 20
for algorithm in algorithms:
    print("Precision@{} of {} algorithm: {}".format(k, algorithm, precision_at_k(test_df, algorithm, k)))